In [1]:
# TODO: move this to an appropriate folder
import inspect
import dill
import heapq # only used for second step of code modification with bindings tracking and in exitlines func

# Notebook Use

In [7]:
debug_messages = True

# Frame definitions and init

In [2]:
class Frame():
    # I, Noemi, refer to this as a FrameNode to remove ambiguity with FrameObjects for the time being. 
    is_global = False
    exitline = None

    def __init__(self, name = None, bindings=None, parent=None, fobj=None):
        self.parent = parent
        self.children = set()
        if bindings is None:
            self.bindings = {}
        self.name = name
        # self's frame object
        self.fobj = fobj

    def bind(self, name, value):
        self.bindings[name] = value # Note: this will allow for duplicate entries. need to figure out a better way to do this. 
    
    def bind(self, name_value_dict):
        self.bindings = name_value_dict # Note: this will allow for duplicate entries. need to figure out a better way to do this. 
    
    def set_name(self, name):
        self.name = name
    
    def add_child(self, child):
        self.children.add(child)
    
    def set_exitline(self, exitline):
        self.exitline = exitline

In [3]:
# a dictionary containing frames with associated variables in the form of a tuple: (type, name, mem loc, value)
env_frames = {"global":{}}
fobj_name_dict = {}
# associates a frame object to a frame node (or in the case of global, at least for the time being, "global" to a frame node.)
fobj_framenode_dict = {}
env_mutables = {}
cnter = 1

# TODO: maybe name "___main___" instead of global? hmmmmmm but if we have wrapper func then no. 
def add_new_frame(name = None, bindings = None, frame_dict = env_frames): # func should always refer to the caller
    curr_frame = inspect.currentframe().f_back
    if bindings is None:
        bindings = {}
    # if no frame nodes have been initialized, set the frame as the global frame and the root
    if len(fobj_framenode_dict.keys()) == 0:
        name = "global"
        fobj_framenode_dict[name] = Frame(bindings = bindings, fobj = curr_frame) # TODO: add fobj
        # OLD: TODO: also current_fobj not listed, fix
        # init FrameNode
        frame = Frame(name = name, bindings = bindings, fobj = curr_frame)
        env_frames[name] = {"name": None, "parent": None, "parent_fobj": None, "curr_fobj": curr_frame, "bindings":bindings}

    else: 
        name = curr_frame.f_code.co_name
        f_name = "f" + name #temp TODO: add number
        parent_frame = curr_frame.f_back
        # TODO: why global be stupid? fix.
        parent = fobj_framenode_dict[parent_frame] if parent_frame in fobj_framenode_dict else fobj_framenode_dict["global"] # frame_name_dict[parent_frame] if parent_frame in frame_name_dict else "global"
        
        # OLD: dictionary rep of frame associations. Use for debugging only. 
        env_frames[f_name] = {"name": name, "parent": parent, "parent_fobj": parent_frame, "curr_fobj": curr_frame, "bindings":bindings}
        # init FrameNode
        frame = Frame(name = name, bindings = bindings, parent = parent, fobj=curr_frame)
        # add self to the pairing dict
        fobj_framenode_dict[curr_frame] = frame
        # modify parent to include self as a child
        parent.add_child(fobj_framenode_dict[curr_frame])
        # OLD: dictionary rep between fobj and frame
        fobj_name_dict[curr_frame] = name
    return frame, fobj_framenode_dict, env_frames, fobj_name_dict # messy

### example code

In [86]:
# TEST: meow
def glob_meow():
    print("in global")
    def A_meow():

        # this is a test message
        z = 5
        return z
    A_meow()
glob_meow()

in global


In [ ]:
# TEST: 
def glob():
    def A():
        z = 7
        #print("AHHHH")
        def B():
            y = 4
            #print("B", dill.detect.varnames(B), inspect.currentframe().f_back)
            def C():
                x = 5
                #print("C", inspect.currentframe().f_back)
                locs = locals()
                variables = {} #(type(locs[var]), var, dill.pointers.at(locs[var]), locs[var]) for var in locs]
                fobj_framenode_dict, env_frames, frame_name_dict = add_new_frame(variables)
                return 8 + x + y + z
            variables = {} #(type(locals()[var]), var, dill.pointers.at(locals()[var]), locals()[var]) for var in locals()]
            fobj_framenode_dict, env_frames, frame_name_dict = add_new_frame(variables)
            return C
        def D():
            #print("C2", inspect.currentframe().f_back)
            locs = locals()
            variables = {var:(type(locs[var]), locs[var]) for var in locs}
            fobj_framenode_dict, env_frames, frame_name_dict = add_new_frame(variables)
            return B()
        #print("A", inspect.currentframe().f_back, inspect.currentframe())
        returnvar = D()()
        locs = locals()
        #variables = [(type(locs[var]), var, dill.pointers.at(locs[var]), locs[var]) for var in locs.keys()]
        variables = {var:(type(locs[var]), locs[var]) for var in locs} # TODO: check whether output of locs[var] is mutable
        fobj_framenode_dict, env_frames, frame_name_dict = add_new_frame(variables) # messy
        print("in A ", inspect.currentframe().f_lineno)
        return returnvar
    cat = A()
#print(fobj_framenode_dict["fA"].f_lineno)
glob()
print(env_frames["fA"]["curr_fobj"].f_lineno)
env_frames

In [38]:
env_frames

{'global': {}}

# code live insertion

## definitions for handling code-insertion

### step 1: initialize FrameNodes and tracing line exits

#### trash code

In [34]:
import sys
from collections import deque

# (self, name = None, bindings=None, parent=None, fobj=None)
class Tracer:
    def __init__(self):
        self.linenos = deque(maxlen=4) # TODO: PLS REFER BACK!!!!! CHANGE maxlen TO 2 IN A PY FILE!!!!

    def trace(self, frame, event, arg):
        if event == 'line' and frame.f_code is not self.log.__code__:
            self.linenos.append(frame.f_lineno)
        return self.trace

    def log(self):
        #print(self.linenos)
        print(self.linenos[0])
        #print(self.linenos)

tracer = Tracer()
sys._getframe().f_trace = tracer.trace
sys.settrace(tracer.trace)

def f():
    if True:
        print(inspect.currentframe().f_lineno)
    else:
        pass
f()
tracer.log()

24
3502


#### actual

In [55]:
example_meow = """
# TEST: meow
def glob_meow():
    print("in global")
    def A_meow():

        # this is a test message
        z = 5
        return z
    A_meow()

glob_meow()"""

In [29]:
example_meow2 = """
# TEST: meow2
def glob_meow2():
    print("in global")
    def A_meow2():

        # this is a test comment
        z = 5
        if True:
            return z

            # test comment
        else:
            return z - 5

    A_meow2()
    
glob_meow2()"""

insert FrameNode tracking into the code

In [30]:
# TODO: do this for lambda as well
# env_mutables
def insertFrameTracking(code):
    # TODO: maybe make it so you can input a func directly
    """ code: code as list or string. 
    this function adds a new line of code initializing the FrameNode in each function definition. """
    # does this by finding a 'def' statement, then inserting the init immediately after
    # if code is given as a string
    if isinstance(code, str):
        code_list = code.split("\n")
    #elif callable(code):
    #    code_list = inspect.getsourcelines(code)[0]
    elif isinstance(code, list):
        code_list = code
    else:
        raise Exception("inputted code not proper format.")

    # TODO: add mutables
    # first init all of the vars
    code_list.insert(0, "env_frames = {'global':{}}")
    code_list.insert(1, "fobj_framenode_dict = {}")
    code_list.insert(2, "fobj_name_dict = {}")

    #  loop through all remaining code.
    i = 3 # modify to consider initalized vars
    while i < len(code_list):
        if code_list[i].lstrip(' ')[:4] == "def ":
            def_line = i
            # find the indent size after the def
            whitespace = 0
            while whitespace == 0:
                i = i + 1
                # check to make sure it's not just an empty line
                if len(code_list[i].split()) > 0:
                    # get size of whitespace before code
                    whitespace = len(code_list[i]) - len(code_list[i].lstrip(' '))
                    # set this back to the previous line just in case the first non-empty line is a def statement.
                    i = i - 1
            # insert the code to create the FrameNode
            # TODO: make sure none of the variables are already in scope in the code.
            code_list.insert(def_line + 1, ' '*whitespace + "frame, fobj_framenode_dict, env_frames, frame_name_dict = add_new_frame() # messy")
        i = i + 1
    # if something was in a define statement, handle that.
    return str.join("\n", code_list)

# TODO: maybe do this in function?
# get modified code with Frame initialization
new_code = insertFrameTracking(example_meow2)
# THEN RUN EXEC(newcode), 
exec(new_code)

in global


End-line tracing

In [31]:
# END-LINE TRACING
def getexitlines():
    # TODO: might get rid of: edits all FrameNodes to include a reference to which line it ends on (in the modified code)
    """edits all FrameNodes to include a reference to which line it ends on in .exitline. 
    Additionally returns a max heap of exit lines, and a dictionary mapping exit lines to Frame objects."""
    if debug_messages: print("========== RUNNING getexitlines ==========") # TODO: make debug_messages a decorator and apply it to all funcs?

    exitlines_pq = []
    exitlines_fobj_dict = {} # might be able to replace this with an exitlines to var list mapping. if so, change description of this func. 
    for fobj in fobj_framenode_dict:
        frame = fobj_framenode_dict[fobj]
        if fobj == "global":
            fobj = frame.fobj
        exitline = frame.fobj.f_lineno - 1
        exitlines_pq.append(exitline)
        exitlines_fobj_dict[exitline] = fobj
        frame.set_exitline(exitline)

    if debug_messages: print("========== COMPLETED getexitlines ==========")
    heapq._heapify_max(exitlines_pq)
    return exitlines_pq, exitlines_fobj_dict
exitlines, exitlines_fobj_dict = getexitlines()

========== RUNNING getexitlines ==========
========== COMPLETED getexitlines ==========


In [189]:
# TEST BLOCK
fobj_framenode_dict[env_frames["fA_meow2"]["curr_fobj"]]
frame_test = env_frames["fA_meow2"]["curr_fobj"]
frame_test.f_code.co_varnames

('fobj_framenode_dict', 'env_frames', 'frame_name_dict', 'z')

### step 2: creating frame objects

In [37]:
# TODO: do this for lambda as well

#X) returnvar = D()()
#X) locs = locals()
#X) bindings = {var:(type(locs[var]), locs[var]) for var in locs} # TODO: check whether output of locs[var] is mutable
#X) frame.bind(bindings) # messy

def insertBindingTracking(exitlines, code):
    """this function inserts tracking for bindings in each function just before it would otherwise end."""
    # keeps track of how many lines have been inserted above the 
    # if code is given as a string
    exitlines = exitlines.copy()
    code_list = code.split("\n")
    
    # get largest exitline
    while len(exitlines) > 0:
        # get the current exitline (also the first line we insert into)!
        exitline = heapq._heappop_max(exitlines)
        i = exitline
        whitespace = len(code_list[i]) - len(code_list[i].lstrip(' '))

        # set variable names for things we are using to store tracking info.
        trk_var_names = ["returnval", "locs", "bindings", "fobj_framenode_dict", "env_frames", "frame_name_dict"]
        # if any of the initial variable names are in the local variable scope, we need to change the offending name(s). TODO: check if this is true once impl is done?
        for k in range(len(trk_var_names)):
            while trk_var_names[k] in exitlines_fobj_dict[exitline].f_code.co_varnames:
                trk_var_names[k] = str(hash(trk_var_names[k] + str(exitline)))[:7]

        if code_list[i].lstrip(' ')[:7] == "return ":
            returnval = code_list[i].lstrip(' ')[7:]
            # set name for returnval in the code
            # change the return line to return the stored value
            code_list[i] = ' '*whitespace + "return " + trk_var_names[0]
            # insert a line that records the return value
            code_list.insert(i, ' '*whitespace + trk_var_names[0] + '=' + returnval)
            i = i + 1
        # store local variables at this point
        code_list.insert(i, ' '*whitespace + trk_var_names[1] + '=' "locals()")
        # store bindings at this point
        # TODO: 'var' might also be an issue here..
        code_list.insert(i + 1, ' '*whitespace + trk_var_names[2] + '=' "{var:(type(" + trk_var_names[1] + "[var]), " + trk_var_names[1] + "[var]) for var in " + trk_var_names[1] + "}")
        # TODO: var names
        code_list.insert(i + 2, ' '*whitespace + "frame.bind(" + trk_var_names[2] + ")")

    return str.join("\n", code_list)

# get modified code with Frame initialization
newnew_code = insertBindingTracking(exitlines, new_code)
# THEN RUN EXEC(newcode), 
exec(new_code)

in global


In [157]:
test = [5, 7, 1]
heapq._heapify_max(test)
#heapq._heappop_max(test)

In [160]:
print(len(test))
heapq._heappop_max(test)

1


1

In [ ]:
# NO LONGER IN USE:

def handle_def(decorator, code_list):
    """this function recursively handles define statements within functions by adding a line at the end of the 
    def that adds some lines of code to the function before it returns."""
    # does this by finding a 'def' statement, then counting the whitespaces of each following line until one has less than or equal to the amount
    # the def func had. Then inserts the new decorator line right before that.
    func_name = None
    def_whitespace_n = 0
    i = 0
    # find the first define statement.
    while not func_name or i >= len(code_list):
        if code_list[i].lstrip(' ')[:4] == "def ":
            func_name = code_list[i].lstrip(' ')[4:].split('(', 1)[0]
            def_whitespace_n = len(code_list[i]) - len(code_list[i].lstrip(' '))
        i = i + 1
    # if something was in a define statement, handle that.
    if func_name:
        # make line that will come after the define statement's end
        new_line = ' '*def_whitespace_n + func_name + '=' + decorator.__code__.co_name + '(' + func_name + ')' + '\n'
        # 
        # TODO: check to make sure this check changes as code_list gets bigger
        #
        while i < len(code_list):
            # check to see if the amount of preceeding whitespace in the define statement is greater than or equal to the amount in the current line
            if def_whitespace_n >= (len(code_list[i]) - len(code_list[i].lstrip(' '))):
                code_list.insert(i, new_line)
                return code_list
            # check to see if a function was defined within the function (if so do recursion)
            elif code_list[i].lstrip(' ')[:4] == "def ":
                # replace that part of the code list with the new code list
                code_list[i - 1:] = handle_def(decorator, code_list[i - 1:]) # i - 1 so we include the def statement
            i = i + 1  
        code_list.append(new_line)  
    return code_list

## definition for A_test and testing block

In [ ]:
def A_test():
    x = 5
    print("x is ", str(x))
    print(inspect.currentframe().f_locals) # !! nestedcode, !!varnames, reference, dir, locals, f_back
    print(dill.detect.varnames(A_test)) # returns a tuple (local vars, local vars referrenced by nested functions) 
    ref = dill.pointers.reference(x)
    print(ref)
    return x
cat = A_test()

x is  5
{'x': 5, 'y': [7, 2, 1, 3]}
(('x', 'y', 'ref'), ())
4629236608


In [12]:
print("OUTPUT BEFORE:")
A_test()
new_code_list = inspect.getsourcelines(A_test)[0]
#new_code_list = lines #handle_def(save_frame, lines)
new_code = "" # new_code_list[0] 
new_code_list[1] = '    print("jadbjhwjd")\n'
#print(new_code_list)
i = 0
while i < len(new_code_list):
    new_code += new_code_list[i]
    #new_code += line + '\n'
    i += 1
#new_code += '\n'
print("new code")
###
#new_code = dill.source.getsource(A, "A")
print(new_code)
#myast = ast.parse(new_code)
#code_obj = compile(myast, 'mymodule', 'exec')
#code_obj = compile(new_code, '<string>', 'exec')
print("OUTPUT:")
exec("""
def A_test():
    print("hello")
y = 7
A()""")

OUTPUT BEFORE:


NameError: name 'A' is not defined

# Trash

In [ ]:
x = 5
def ahhh(y):
    x = y + 4
    #print(dir())
    #print(inspect.trace())
    #print(inspect.stack())
    print(inspect.currentframe().f_back)
    print(inspect.currentframe())
    return 8
#print(globals())
#print(dir())
#print(locals())
#ahhh(8)
#traceback.extract_tb()
#frame_infos = inspect.trace()
#print(inspect.trace())
#ahhh(8)
#print(len(inspect.stack()))
#print(inspect.stack())
#print(inspect.currentframe())
#aaa = inspect.getframeinfo(inspect.currentframe())
#print(type(aaa))
#print(aaa)
#print(inspect.getinnerframes(aaa))
stack = inspect.stack()
outer_frames = inspect.getouterframes(inspect.currentframe())
inner_frames = [frame for frame in stack if frame not in outer_frames]
print(inner_frames)
#print(current_frame)

In [ ]:
import inspect
import copy
import types
import dill


#Define a function we want to modify:
def test():
    print("Here")

#Run the function to check output
print("\n\nRunning Function...")
test()
#>>> Here

#Get the source code for the test function:
testSource2 = inspect.getsourcelines(test)[0]#
testSource = dill.source.getsource(test)
print("\n\ntestSource:")
print(testSource)


#Take the inner part of the source code and modify it how we want:
newtestinnersource = ''
testSourceLines = testSource.split('\n')
print("lines")
print(testSourceLines)
linenumber = 0 
for line in testSourceLines:
    if (linenumber > 0):
        if (len(line[4:]) > 0):
            newtestinnersource += line[4:] + '\n'
    linenumber += 1
#newtestinnersource += 'print("Here2")\n'
newtestinnersource += 'return 2\n'
#print("\n\nnewtestinnersource")
print("new")
print(newtestinnersource)


#Re-assign the function's code to be a compiled version of the `innersource`
code_obj = compile(newtestinnersource, '<string>', 'exec')
test.__code__ = copy.deepcopy(code_obj)
print("\n\nRunning Modified Function...")
a = test() #<- NOW HAS MODIFIED SOURCE CODE, AND PERFORMS NEW TASK
#>>>Here
#>>>Here2
testSource2
test.__code__
print("a:")
print(a)


In [ ]:
x = 5
y = x
x = 8
print(y)

5


# typing casting to try to get a valid Traceback

In [ ]:
import types
import typing

In [ ]:
"""FrameInfo(frame=<frame at 0x106e498a0, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_1031/310886205.py', line 19, code <module>>, filename='/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_1031/310886205.py', lineno=19, function='<module>', code_context=['stack = inspect.stack()\n'], index=0, positions=Positions(lineno=19, end_lineno=19, col_offset=8, end_col_offset=23)),"""
test2 = typing.cast(types.TracebackType, inspect.getframeinfo(inspect.currentframe()))
test2.f_lineno = test2.tb_lineno
#test = types.TracebackType(test2, inspect.currentframe(), 0, 19)


AttributeError: 'Traceback' object has no attribute 'tb_lineno'

In [ ]:
type(test2)
inspect.getinnerframes(typing.cast(types.TracebackType, inspect.getframeinfo(inspect.currentframe())))

In [ ]:
traceback.format_tb(inspect.getframeinfo(inspect.currentframe()))

AttributeError: 'Traceback' object has no attribute 'tb_frame'